# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4997, uptime 0:00:17
mini_beamline                    RUNNING   pid 4998, uptime 0:00:17
random_walk                      RUNNING   pid 4999, uptime 0:00:17
random_walk_horiz                RUNNING   pid 5000, uptime 0:00:17
random_walk_vert                 RUNNING   pid 5001, uptime 0:00:17
simple                           RUNNING   pid 5002, uptime 0:00:17
thermo_sim                       RUNNING   pid 5003, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 5004, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 01:18:51
Persistent Unique Scan ID: 'b0cd14be-bcb4-42a8-9372-fc836a70ae6c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 01:18:51.3 |     -1.000 |     102517 |


|         2 | 01:18:51.3 |     -0.500 |     103980 |
|         3 | 01:18:51.3 |      0.000 |     104595 |
|         4 | 01:18:51.4 |      0.500 |     103264 |
|         5 | 01:18:51.4 |      1.000 |     101593 |


+-----------+------------+------------+------------+
generator scan ['b0cd14be'] (scan num: 1)





('b0cd14be-bcb4-42a8-9372-fc836a70ae6c',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/b0cd14be-bcb4-42a8-9372-fc836a70ae6c-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 01:18:51
Persistent Unique Scan ID: '4096dbc3-1e09-4dae-b03d-a9fa7154ecc0'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 01:18:51.8 |          -1 | 1105168781 |


|         2 | 01:18:52.0 |          -0 | 1098591344 |


|         3 | 01:18:52.1 |           0 | 1086189816 |
|         4 | 01:18:52.2 |           0 | 1078905543 |


|         5 | 01:18:52.3 |           1 | 1073149037 |


+-----------+------------+-------------+------------+
generator scan ['4096dbc3'] (scan num: 2)





('4096dbc3-1e09-4dae-b03d-a9fa7154ecc0',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 01:18:52
Persistent Unique Scan ID: '198cf2a7-2df1-4171-a716-56c5ece7162c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 01:18:52.8 | 1056713093 |


|         2 | 01:18:53.7 | 1078301978 |


|         3 | 01:18:54.7 | 1129214455 |


...

|        28 | 01:19:19.8 | 1104347384 |


|        29 | 01:19:20.8 | 1054956617 |


|        30 | 01:19:21.8 | 1081254500 |


+-----------+------------+------------+
generator count ['198cf2a7'] (scan num: 3)





('198cf2a7-2df1-4171-a716-56c5ece7162c',)